In [34]:
# Firstly, we import the necessary libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2 
import pandas as pd
import os

In [88]:
# The first task will be to change the csv file
# Change 1: Replace C/.... in a format so that it can be read from google drive
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
# Specify the column names for the dataframe
df = pd.read_csv('driving_log.csv', names = columns)
# To view only 9 rows
pd.set_option('display.max_rows', 9)
# This will remove the unecessary prefixes
# pop() returns the item present at the given index and also deletes that value from the list
for i in range(1248):
    for j in range(3):
        df.iloc[i, j] = '/content/gdrive/My Drive/IMG/' + df.iloc[i, j].split('C:\\Users\\Sayan\\Desktop\\Udacity self driving car project\\IMG\\').pop(1)
# Save the changes made in the csv file        
df.to_csv('driving_log.csv')

IndexError: pop index out of range

In [97]:
# Remove the columns which are not required
df.drop([ 'throttle', 'reverse', 'speed'], axis = 1, inplace = True)

KeyError: "['throttle' 'reverse' 'speed'] not found in axis"

In [107]:
# Remove any unwanted row using row-number
df = df.iloc[1:]
df

,center,left,right,steering
1.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.0
2.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.0
3.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,-0.2
4.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,-0.35
...,...,...,...,...
1244.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.0
1245.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.0
1246.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.0
1247.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.0


In [109]:
df.to_csv('driving_log.csv')

In [117]:
# Steering angle for left image will be steering angle + 0.4 and right image will be steering angle - 0.4
df['steering'].tolist()
stabs = []
for st in df['steering'].tolist():
    st = float(st)
    stabs.append(st)

In [120]:
# Insert float values of the steering angles (Conversion from string to float)
df.insert(3, "steering_center", stabs, True)

In [122]:
# Insert steering angle values for left and right camera images by adding +0.4 and -0.4
sl = []
sr = []
for st in stabs:
    sl.append(st + 0.4)
    sr.append(st - 0.4)

In [123]:
df.insert(4, "steering_left", sl, True)
df.insert(5, "steering_right", sr, True)

In [126]:
# Remove steering column to avoid confusion
df.drop(['steering'], axis = 1, inplace = True)
df

KeyError: "['steering'] not found in axis"

In [128]:
df.to_csv('driving_log.csv')

In [129]:
df

,center,left,right,steering_center,steering_left,steering_right
1.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.00,0.40,-0.40
2.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.00,0.40,-0.40
3.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,-0.20,0.20,-0.60
4.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,-0.35,0.05,-0.75
...,...,...,...,...,...,...
1244.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.00,0.40,-0.40
1245.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.00,0.40,-0.40
1246.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.00,0.40,-0.40
1247.0,/content/gdrive/My Drive/IMG/center_2020_01_21...,/content/gdrive/My Drive/IMG/left_2020_01_21_1...,/content/gdrive/My Drive/IMG/right_2020_01_21_...,0.00,0.40,-0.40
